In [4]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"@", "", string)
    return string.lower()

In [5]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1111806

In [2]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re
import jsonlines

import sys
import os

os.environ['KERAS_BACKEND'] = 'theano'

from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers.merge import concatenate
from keras.layers import Add

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input

from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
import tensorflow as tf
from config import Config
from cnn_model import cnn_model

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
count = 0
train_val_data = []
test_data = []
with jsonlines.open('instances.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        count += 1
        if (count > 9275):
            test_data.append(obj)
        if(count<=9275):
            train_val_data.append(obj)

count = 0
truth_data = []
with jsonlines.open('truth.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        truth_data.append(obj)


In [7]:
def imgModel(vals_df):
    # model = VGG16(weights='imagenet', include_top=False)
    # model.summary()
    img_features = []
    config = Config()
    images = tf.placeholder(
        dtype=tf.float32,
        shape=[config.batch_size] + config.image_shape)

    sess = tf.Session()

    model = cnn_model(config)
    features = model.build_vgg16(images)
    model.load_cnn(sess,config.vgg16_file)

    for entry in vals_df.values:
        img_path = entry[1][0]
        # print(img_path)
        img = image.load_img(img_path, target_size=(224, 224,3))
        img_data = image.img_to_array(img)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        imgList = img_data

        # vgg16_feature = model.predict(img_data)

        vgg16_feature = sess.run(features,feed_dict={images:img_data})
        # print("----------------------------------------------------------")
        # print(type(vgg16_feature))
        img_features.append(vgg16_feature[0])

    return img_features


In [37]:
features = ["id", "postMedia", "targetTitle", "truthClass"]
truth_data_df = pd.DataFrame.from_dict(truth_data)

test_data_df = pd.DataFrame.from_dict(test_data)
test = pd.merge(test_data_df, truth_data_df, on="id")

final_vals = []
tdata = test[features]
tdata = tdata.values.tolist()
for i in range(len(tdata)):
    if tdata[i][1] != []:
        final_vals.append([tdata[i][0], [tdata[i][1][0]], [tdata[i][2]], tdata[i][3]])

vals_df = pd.DataFrame(final_vals, columns=["id", "file_path", "title", "truthClass"])
image_features_test = imgModel(vals_df)
tdata = test[features].values
# labels = []
tlabels = []
df = []

for i in test.values:
    if (i[9] == "clickbait"):
        tlabels.append(1)
    else:
        tlabels.append(0)

ValueError: Variable conv1_1/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/ashwini/PycharmProjects/Research/nn.py", line 70, in conv2d
    name = name)
  File "/home/ashwini/PycharmProjects/Research/cnn_model.py", line 53, in build_vgg16
    conv1_1_feats = self.nn.conv2d(images, 64, name = 'conv1_1')
  File "<ipython-input-7-d8008e7e96d2>", line 13, in imgModel
    features = model.build_vgg16(images)


In [38]:
t_df = []
tdata = vals_df

for i in range(tdata.values.shape[0]):
    text = []
    for j in range(1,5):
        k = tdata.values[i][2]
        text+=(k)
    words = ""
    for string in text:
        string = clean_str(string)
        words +=" ".join(string.split())
    t_df+=[words]


In [19]:
tokenizer_test = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_test.fit_on_texts(t_df)
test_sequences = tokenizer_test.texts_to_sequences(t_df)

word_index = tokenizer_test.word_index
tdata = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
tlabels = to_categorical(np.asarray(tlabels))

x_test = tdata
y_test = tlabels
print("X_test ", x_test.shape)

X_test  (5463, 1000)


In [46]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) +1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) +1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)


Total 400000 word vectors.


In [39]:
train_data_df = pd.DataFrame.from_dict(train_val_data)
train = pd.merge(train_data_df, truth_data_df, on="id")
data = train[features].values #TRAIN + VALID
print("Data shape ", data.shape)

Data shape  (9275, 4)


In [40]:
df = []
for i in range(data.shape[0]):
    text = []
    for j in range(0,5):
        k = data[i][2]
        text+=k
    words = ""
    for string in text:
        string = clean_str(string)
        words +=" ".join(string.split())
    df+=[words]

In [41]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df)
word_index = tokenizer_test.word_index

In [45]:
image_data_input = Input(shape=(100352,), dtype='float32')
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds_text = Dense(2, activation='softmax')(l_lstm)

preds_image = Dense(2, activation='softmax')(image_data_input)

preds_add = Add()([preds_text, preds_image])

preds = Dense(2)(preds_add)

model = Model([sequence_input, image_data_input], preds)
model.load_weights('weights-bilstm-03-0.22.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print("model fitting - multimodal Bidirectional LSTM")
model.summary()
preds = model.predict([x_test, np.asarray(image_features_test)], batch_size=50, verbose=1)
preds = preds.round()
print(accuracy_score(y_test, preds))

/usr/local/lib/python3.5/dist-packages/keras/layers/recurrent.py:2024: UserWarning: RNN dropout is no longer supported with the Theano backend due to technical limitations. You can either set `dropout` and `recurrent_dropout` to 0, or use the TensorFlow backend.
  'RNN dropout is no longer supported with the Theano backend '


model fitting - multimodal Bidirectional LSTM
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1000, 100)    1574000     input_20[0][0]                   
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, 200)          160800      embedding_5[0][0]                
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 100352)       0                                            
_______________________________________________________________

IndexError: index 15740 is out of bounds for size 15740
Apply node that caused the error: AdvancedSubtensor1(embedding_4/embedding_5/embeddings, Reshape{1}.0)
Toposort index: 44
Inputs types: [TensorType(float32, matrix), TensorType(int32, vector)]
Inputs shapes: [(15740, 100), (50000,)]
Inputs strides: [(400, 4), (4,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{3}(AdvancedSubtensor1.0, MakeVector{dtype='int64'}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/ashwini/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2662, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/home/ashwini/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2785, in _run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/ashwini/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast

In [25]:
print(len(image_features_test))

5463
